In [0]:
%pip install uv databricks-agents
%restart_python

In [0]:
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext)

In [0]:
import os
from IPython.display import Image, display
import mlflow
from mlflow.models.resources import DatabricksServingEndpoint, DatabricksVectorSearchIndex
from databricks.agents import deploy
from langchain_core.messages import HumanMessage

from agents.supervisor.graph import app as supervisor

In [0]:
DATABRICKS_HOST = dbutils.secrets.get(scope="mlc_credentials", key="databricks_host")
DATABRICKS_TOKEN = dbutils.secrets.get(scope="mlc_credentials", key="databricks_token")

os.environ["DATABRICKS_HOST"] = DATABRICKS_HOST
os.environ["DATABRICKS_TOKEN"] = DATABRICKS_TOKEN

messages = {"messages": [{"role": "user", "content": "What is Apache Spark?"}]}

In [0]:
genie_response['messages'][3]

In [0]:
messages[0]['messages'][0]

In [0]:
messages[1]

In [0]:
request = {"messages": [{"role": "user", "content": "Which plant to distribution center route is running the most product?"}]}

messages = []
for event in supervisor.stream(request, stream_mode="updates"):
    for node_data in event.values():
        # Supervisor router messages return None
        if node_data is not None:
            messages.extend(
                ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
            )

In [0]:


messages = []
for event in supervisor.stream(request, stream_mode="updates"):
  for node_data in event.values():
    # Supervisor router messages return None
    if node_data is not None:
      messages.append(node_data)

In [0]:
for node_data in genie_response:
  for node_data in event.values():
            # Supervisor router messages return None
            if node_data is not None:
  print(node_data.values())

In [0]:

genie_response = []
for event in supervisor.stream({"messages": [{"role": "user", "content": "Which plant to distribution center route is running the most product?"}]}, stream_mode="updates"):
  genie_response.append(event)

In [0]:
supervisor.invoke(messages)

In [0]:
from chatagent import AGENT

In [0]:
messages = {"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages?"}]}

AGENT.predict(messages)

In [0]:
config = mlflow.models.ModelConfig(development_config='config.yaml')

supervisor_llm = config.get("agents").get("supervisor")[0]['llm']
genie_llm = config.get("agents").get("genie")[0]['llm']

rag_config = config.get("agents").get("rag")[0]
rag_llm = rag_config['llm']
index_location = rag_config['index_location']

llms = list(set([supervisor_llm, genie_llm,  rag_llm]))
serving_endpoints = [DatabricksServingEndpoint(endpoint_name=llm) for llm in llms]
mlflow_config = config.get("mlflow")
experiment = mlflow_config['experiment_location']
uc_model = mlflow_config['uc_model']
input_example = mlflow_config['input_example']

mlflow.set_experiment(experiment)
mlflow.set_registry_uri("databricks-uc")

In [0]:
AGENT.predict(input_example)

In [0]:
with mlflow.start_run(run_name="example_agent"):

  model_info = mlflow.pyfunc.log_model(
                  python_model = "chatagent.py",
                  streamable=True,
                  model_config="config.yaml",
                  artifact_path="graph",
                  input_example=input_example,
                  code_paths = [
                    'agents',
                    'graph.py'
                    ],
                  resources = [
                    DatabricksVectorSearchIndex(index_name=index_location),
                    *serving_endpoints
                    ],
                  pip_requirements = "requirements.txt"
               )
  
  mlflow.log_artifact("graph.png")

  model_uri = model_info.model_uri

  loaded_app = mlflow.pyfunc.load_model(model_uri)
  loaded_app.predict(input_example)
  
print(model_uri)

### Verify deployment

In [0]:
messages = {"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages?"}]}

mlflow.models.predict(
    model_uri=model_uri,
    input_data=messages,
    env_manager="uv",
)

In [0]:
import mlflow 

#model_uri = 'runs:/f9ba24a55bee4016b2514ca5fd566f51/graph'

pyfunc_model = mlflow.pyfunc.load_model(model_uri)
input_data = pyfunc_model.input_example

mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="uv",
)

In [0]:
model_uri

In [0]:
model_info = mlflow.register_model(model_uri, 
                                   name = uc_model)

In [0]:
deployment_info = deploy(model_name=uc_model, 
                         model_version=model_info.version,
                         #model_version = 3,
                         environment_vars = {"DATABRICKS_HOST": "{{secrets/mlc_credentials/databricks_host}}",
                                             "DATABRICKS_TOKEN": "{{secrets/mlc_credentials/databricks_token}}"})

In [0]:
import os
import requests
import numpy as np
import pandas as pd
import json

host = dbutils.secrets.get('mlc_credentials', 'databricks_host')
token = dbutils.secrets.get('mlc_credentials', 'databricks_token')
url = f'{host}/serving-endpoints/agents_main-default-langgraph_example_agent/invocations'

def score_model(dataset):
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    data_json = json.dumps(dataset, allow_nan=True)
    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    return response.json()

In [0]:
input_example

In [0]:
score_model(input_example)

In [0]:

{"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages?"}]}
score_model(messages)